In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Parkinson/

In [ ]:
import os
import pandas as pd
import numpy as np
import random, re

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)

seed_everything()

### 39개 관측 센서 위치

In [ ]:
BODY = ['LFHD', 'RFHD', 'LBHD', 'RBHD',  # 머리 앞뒤
        'CLAV',  # 쇄골
        'RBAK',  # 어깨뼈
        'C7',    # 경추 7번
        'STRN',  # 흉골(명치)
        'T10',   # 흉추 10번
        ]

ARMS = ['LSHO', 'RSHO',  # 어깨
        'LUPA', 'RUPA',  # 위팔
        'LELB', 'RELB',  # 팔꿈치
        'LFRM', 'RFRM',  # 아래팔
        'LWRA', 'RWRA', 'LWRB', 'RWRB',  # 손목
        'LFIN', 'RFIN',  # 손가락
        ]

LEGS = ['LASI', 'RASI', 'LPSI', 'RPSI',  # 골반 앞뒤
        'LTHI', 'RTHI',  # 대퇴
        'LKNE', 'RKNE',  # 무릎
        'LTIB', 'RTIB',  # 정강이
        'LHEE', 'RHEE',  # 발뒤꿈치
        'LANK', 'RANK',  # 복사뼈
        'LTOE', 'RTOE',  # 발가락
        ]

ALL = BODY + ARMS + LEGS
len(ALL)

### 보유데이터 정보 불러오기

In [ ]:
df_info = pd.read_csv("csv_info.csv")
df_info

In [ ]:
df_info['Category'].value_counts()

csv를 읽을 때 [ ] 를 문자열로 인식하니까...리스트로 저장해도 리스트로 읽을 수 없구나..

In [ ]:
type(df_info['pathFW'][0])

### 환자 데이터 랜덤추출

In [ ]:
ptype = 'PD'
n_sample = 1
walk_direction = 'FW'

In [ ]:
def dataSampling(df, ptype, n_sample, walk_direction):
    sampled_df = df.loc[df['Category']==ptype].sample(n_sample)
    
    sampled_paths = []  # 랜덤추출한 참여자별 FW/BW 데이터 하나씩 선택
    for id in sampled_df.index:
        paths = list(map(lambda s: s.strip(), re.sub("'", "", sampled_df.loc[id, 'path'+walk_direction][1:-1]).split(',')))
        sampled_paths.extend(random.sample(paths, 1))
    
    return sampled_paths, list(sampled_df['Participant']), 

In [ ]:
sampled_paths, _ = dataSampling(df_info, ptype, n_sample, walk_direction)
sampled_paths

In [ ]:
name, walk_order = sampled_paths[0][:-4].split('/')[-1].split('_')
print(f'[Participant INFO] Name: {name}, Type: {ptype}, Trial: {walk_order}')

df = pd.read_csv(sampled_paths[0])
df.rename(columns={"Unnamed: 0": "Frame"}, inplace=True)
df

### 시간(프레임)에 따른 X, Y, Z좌표 관찰

In [ ]:
def plotByTime(df, placement: str):  
    '''
    placement: 관찰하고 싶은 마커 위치
    '''

    fig, axes = plt.subplots(figsize=(10,3), ncols=3)
    plt.subplots_adjust(wspace=0.4)

    for i, direction in enumerate(['X','Y','Z']):
        target = placement + '_' + direction
        sns.lineplot(data=df, x='Frame', y=target, ax=axes[i])
        sns.set_style('darkgrid')

    plt.suptitle(f'{placement}_X,Y,Z values in {walk_order} trial of {name}', fontsize=14)
    plt.show()

In [ ]:
for placement in BODY:
    plotByTime(df, placement)

### 상반신 흔들림/진동 관찰